# 1. RUN THESE FIRST

In [ ]:
from __future__ import print_function, division
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

# SKIP

In [ ]:
year_list = [x for x in range(2018,2016,-1)]
court_list = ['frontcourt','backcourt']
conference_list = ['eastern','western',]

In [ ]:
# https://www.basketball-reference.com/allstar/NBA_2017_voting-frontcourt-eastern-conference.html
concat_list = []
for year in year_list:
    for court in court_list:
        for conference in conference_list:
            url = 'https://www.basketball-reference.com/allstar/NBA_{sub1}_voting-{sub2}-{sub3}-conference.html'
            response = requests.get(url.format(sub1 =year, sub2 =court,sub3 =conference))
            page = response.text
            soup = BeautifulSoup(page, "lxml")
            text_all=''
            # scrapes html from website with player name and votes
            all = soup.find_all(
                   lambda tag: tag.name == 'td' and (tag['data-stat'] == 'player_name' or tag['data-stat'] == 'fan_votes')
               )

            #takes text portion of html and cleans data using RegEx
            for i in range(len(all)):
                text_all += all[i].text
                string = re.sub(r'([a-zA-Z])(\d)', r'\1,\2',text_all)
                string = re.sub(r'(\d)([a-zA-Z])', r'\1,\2',string)
                string = re.sub(r'(\d),(\d)', r'\1\2', string)
                string = re.sub(r'([a-zA-Z]\.)(\d)', r'\1,\2',string)

            # creates list with player names and votes as separate elements
            all_list = string.split(",")

            # creates a list of list, combining player name and votes into one list, and inserting year as well
            new_list = []
            it = iter(all_list)
            for x in it:
                new_list.append([x, int(next(it))])
            for player_vote in new_list:
                player_vote.insert(0,year)
            df = pd.DataFrame(new_list, columns=['Year', 'Player','Votes'])
            concat_list.append(df)
            
df_recent = pd.concat(concat_list, axis=0)

In [ ]:
# import pickle
# filename = '2018_votes_pickle'
# outfile = open(filename,'wb')
# pickle.dump(df_recent,outfile)
# outfile.close()

# 2. RUN

In [ ]:
import pickle
filename = '2018_votes_pickle'
infile = open(filename,'rb')
df_recent = pickle.load(infile)
infile.close()

# SKIP

In [ ]:
lists_of_stats = []

for year in year_list:
    for court in court_list:
        for conference in conference_list:
            url = 'https://www.basketball-reference.com/allstar/NBA_{sub1}_voting-{sub2}-{sub3}-conference.html'
            response = requests.get(url.format(sub1 =year, sub2 =court,sub3 =conference))
            page = response.text
            soup = BeautifulSoup(page, "lxml")
            all = soup.find_all(
                       lambda tag: tag.name == 'td' and (tag['data-stat'] == 'player_name' or tag['data-stat'] == 'fan_votes'))
            index_all = [x for x in range(0,len(all),2)]
            list_links = []
            for i in index_all:
                all = soup.find_all(
                       lambda tag: tag.name == 'td' and (tag['data-stat'] == 'player_name' or tag['data-stat'] == 'fan_votes')
                   )[i].find('a')['href']
                list_links.append(all)

            for stats_link in list_links:
                url_stats_new = 'https://www.basketball-reference.com{}'
                response_stats_new = requests.get(url_stats_new.format(stats_link))
                page_stats_new = response_stats_new.text
                soup_stats_new = BeautifulSoup(page_stats_new, "lxml")
                pergame = soup_stats_new.find('tr', id =('per_game.'+ str(year)))
                pergame_diff = soup_stats_new.find('tr', id =('per_game.'+ str(year-1))) #added
                if pergame:
                    find_pergame = pergame.find_all(
                        lambda tag: tag.name == 'td' and tag['data-stat'])
                    stat_list = []
                    for stat in range(len(find_pergame)):
                        stat_list.append(find_pergame[stat].text)
                    
                    if pergame_diff: # added
                        find_pergame_diff = pergame_diff.find_all(
                        lambda tag: tag.name == 'td' and tag['data-stat'])
                        stat_list.append(find_pergame_diff[-1].text)
                    else:
                        stat_list.append("None")
                                                             
                    if pergame.find_all(class_ = "sr_star"):
                        stat_list.append(1)
                    else:
                        stat_list.append(0)
                else:
                    stat_list = [0]*30
            #         stat_list.insert(0,f + " " + l)
                stat_list.insert(0,year)
                lists_of_stats.append(stat_list)

In [ ]:
df_stats = pd.DataFrame(lists_of_stats, columns=['Year', 'Age','Team','League','Pos','GP','GS', 'MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','Prev_PTS','Star',])

In [ ]:
# import pickle
# filename3 = '2018_stats_pickle'
# outfile3 = open(filename3,'wb')
# pickle.dump(df_stats,outfile3)
# outfile3.close()

# 3. Start Here

In [ ]:
import pickle
filename3 = '2018_stats_pickle'
infile = open(filename3,'rb')
df_2018_stats = pickle.load(infile)
infile.close()

In [ ]:
new_index = [x for x in range(0,963)]
df_recent_r = df_recent.reset_index()
df_recent_r['index'] = new_index
df_recent_r.head()

In [ ]:
df_2018_stats_r = df_2018_stats.reset_index()
df_2018_stats_r.head()

In [ ]:
df_merge_2018 = df_recent_r.merge(df_2018_stats_r, how='left',on=['index','Year'])
df_merge_2018 = df_merge_2018.drop(columns = 'index')
df_merge_2018.head()

# ALL COMBINE WITH 2016

In [ ]:
import pickle
filename2 = 'df_2016'
infile = open(filename2,'rb')
df_2016 = pickle.load(infile)
infile.close()

In [ ]:
df_all = pd.concat([df_merge_2018,df_2016],ignore_index=True)

In [ ]:
df_all.groupby('Player', as_index=False).Star.sum().sort_values('Star', ascending=False).head()

In [ ]:
reversed_df = df_all.iloc[::-1]

In [ ]:
reversed_df['Cum_Star'] = reversed_df.groupby(['Player'])['Star'].cumsum()

In [ ]:
df_all = reversed_df.iloc[::-1]

In [ ]:
df_all.columns = ['Year', 'Player', 'Votes', 'Age', 'Team', 'League', 'Pos', 'GP', 'GS',
       'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
       'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'PTS_PREV', 'Star', 'Cum_Star']

In [ ]:
# import pickle
# filename5 = 'df_all'
# outfile5 = open(filename5,'wb')
# pickle.dump(df_all,outfile5)
# outfile5.close()